<a href="https://colab.research.google.com/github/UtwoA/Introduction_to_ML/blob/main/ml6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [52]:
data = pd.read_csv("Supermarket.txt", sep="\t", skiprows=1, names=["Чек", "Товар"], encoding="cp1251")

In [53]:
basket = data.groupby("Чек")["Товар"].agg(list)
transaction_list = basket.tolist()

enc = TransactionEncoder()
matrix = enc.fit(transaction_list).transform(transaction_list)
df_bin = pd.DataFrame(matrix, columns=enc.columns_)

In [54]:
freq_sets = apriori(df_bin, min_support=0.01, use_colnames=True)

In [55]:
rules = association_rules(freq_sets, metric="confidence", min_threshold=0.3)
rules = rules.sort_values("confidence", ascending=False)

print(f"Найдено правил ассоциаций: {len(rules)}")
print("\nТоп-10 правил по достоверности:")
print(rules.head(10), "\n")

Найдено правил ассоциаций: 192

Топ-10 правил по достоверности:
                                           antecedents  \
31                     (ВАФЛИ, КЕТЧУПЫ, СОУСЫ, АДЖИКА)   
52                                       (ВАФЛИ, СЫРЫ)   
40                                        (ВАФЛИ, МЕД)   
32                         (ВАФЛИ, МАКАРОННЫЕ ИЗДЕЛИЯ)   
153                    (СЫРЫ, МАКАРОННЫЕ ИЗДЕЛИЯ, МЕД)   
156                 (СЫРЫ, МАКАРОННЫЕ ИЗДЕЛИЯ, СУХАРИ)   
157                  (ЧАЙ, МАКАРОННЫЕ ИЗДЕЛИЯ, СУХАРИ)   
160                                (СЫРЫ, МЕД, СУХАРИ)   
189                (СЫРЫ, МЕД, КЕТЧУПЫ, СОУСЫ, АДЖИКА)   
182  (СЫРЫ, МАКАРОННЫЕ ИЗДЕЛИЯ, МЕД, КЕТЧУПЫ, СОУСЫ...   

                   consequents  antecedent support  consequent support  \
31                       (ЧАЙ)            0.022727            0.750000   
52                       (ЧАЙ)            0.113636            0.750000   
40                       (ЧАЙ)            0.136364            0.750000   
3

Чем ниже support, тем больше наборов обнаруживается, но среди них много нерелевантных и редких.

При высоком confidence остаются только самые надёжные закономерности.

Баланс параметров даёт лучшее качество (например min_support=0.02, confidence=0.5).

In [56]:
waffle_related = rules[rules['antecedents'].apply(lambda combo: "ВАФЛИ" in combo)]

if not waffle_related.empty:
    best = waffle_related.iloc[waffle_related["confidence"].idxmax()]
    item = list(best["consequents"])[0]
    print(f"Самый связанный товар с вафлями: {item} (confidence = {best['confidence']:.2f})")
else:
    print("Товаров, связанных с вафлями, не обнаружено.")

Самый связанный товар с вафлями: СЫРЫ (confidence = 0.50)


Какой товар чаще всего покупают с вафлями

По построенным правилам было найдено правило с максимальной confidence, где в левой части присутствует "ВАФЛИ".
Товар, который чаще всего берут вместе с вафлями — СЫРЫ

In [57]:
honey_cheese = [check for check in transaction_list if ("МЕД" in check) and ("СЫРЫ" in check)]

if honey_cheese:
    items_present = set().union(*honey_cheese)
    all_goods = set(df_bin.columns)
    excluded = list(all_goods - items_present - {"МЕД", "СЫРЫ"})

    popularity = df_bin.sum().sort_values(ascending=False)

    for prod in popularity.index:
        if prod in excluded:
            print(f"Вероятно НЕ купят вместе с медом и сыром: {prod}")
            break
    else:
      print("Нет транзакций, где одновременно брали мёд и сыр.")

Нет транзакций, где одновременно брали мёд и сыр.


Были выделены все чеки, где одновременно содержались МЁД и СЫРЫ.
На их основе найдены товары, которые ни разу не встречались одновременно с данной парой — то есть вероятность покупки минимальна.

In [58]:
top_sets = freq_sets.sort_values("support", ascending=False).head(5)
print("\nТоп-5 самых популярных товарных наборов:")
for _, row in top_sets.iterrows():
    goods = ", ".join(list(row['itemsets']))
    print(f"{goods} — support = {row['support']:.4f}")


Топ-5 самых популярных товарных наборов:
ЧАЙ — support = 0.7500
МАКАРОННЫЕ ИЗДЕЛИЯ — support = 0.5455
КЕТЧУПЫ, СОУСЫ, АДЖИКА — support = 0.5227
МЕД — support = 0.5000
МАКАРОННЫЕ ИЗДЕЛИЯ, КЕТЧУПЫ, СОУСЫ, АДЖИКА — support = 0.4545


In [59]:
best_lift_rules = rules.sort_values("lift", ascending=False).head(5)
print("\n 5 правил:")
for _, r in best_lift_rules.iterrows():
    A = ", ".join(list(r['antecedents']))
    B = ", ".join(list(r['consequents']))
    print(f"Если купили [{A}] → вероятнее купят [{B}]")
    print(f"Support={r['support']:.3f}, Confidence={r['confidence']:.3f}, Lift={r['lift']:.2f}\n")


 5 правил:
Если купили [СЫРЫ, МАКАРОННЫЕ ИЗДЕЛИЯ, МЕД] → вероятнее купят [ЧАЙ, КЕТЧУПЫ, СОУСЫ, АДЖИКА]
Support=0.045, Confidence=1.000, Lift=3.38

Если купили [СЫРЫ, МЕД, КЕТЧУПЫ, СОУСЫ, АДЖИКА] → вероятнее купят [ЧАЙ, МАКАРОННЫЕ ИЗДЕЛИЯ]
Support=0.045, Confidence=1.000, Lift=3.38

Если купили [ВАФЛИ, МАКАРОННЫЕ ИЗДЕЛИЯ] → вероятнее купят [СУХАРИ]
Support=0.023, Confidence=1.000, Lift=3.14

Если купили [ВАФЛИ, МЕД, ЧАЙ] → вероятнее купят [СЫРЫ, СУХАРИ]
Support=0.045, Confidence=0.333, Lift=2.93

Если купили [ВАФЛИ, МЕД] → вероятнее купят [ЧАЙ, СЫРЫ, СУХАРИ]
Support=0.045, Confidence=0.333, Lift=2.93

